In [ ]:
import pystan
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import arviz as az

In [ ]:
#### Preprocecing ###

In [ ]:
jh_dir = "../../COVID-19"

In [ ]:
Confirmed = pd.read_csv(jh_dir + "/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

In [ ]:
df = Confirmed.groupby("Country/Region").sum()
df = df.drop(["Lat", "Long"], 1)
df = df.T
df = df.set_index(pd.to_datetime(df.index))
confirmed = df
confirmed = confirmed.cummax()

In [ ]:
df = pd.read_csv(jh_dir + "/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
df = df.groupby("Country/Region").sum()
df = df.drop(["Lat", "Long"], 1)
df = df.T
df = df.set_index(pd.to_datetime(df.index))
recovered = df.cummax()

In [ ]:
df = pd.read_csv(jh_dir + "/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
df = df.groupby("Country/Region").sum()
df = df.drop(["Lat", "Long"], 1)
df = df.T
df = df.set_index(pd.to_datetime(df.index))
death = df.cummax()

In [ ]:
### Data Preparation ###

In [ ]:
country = 'Japan'
epoch = pd.to_datetime('2020-01-22')
last = pd.to_datetime('2020-04-18')
C0 = confirmed.loc[epoch:last, country].values
R0 = (recovered).loc[epoch:last, country].values
D0 = death.loc[epoch:last, country].values
P = 12000_0000
iteration=10_0000
repeat = 1000

In [ ]:
data = {'T': C0.shape[0], 'T0': C0.shape[0], 'P': P, 'C0': C0, 'R0': R0, 'D0':D0}

In [ ]:
### Model ###

In [ ]:
sm_1 = pystan.StanModel(file="single-step.stan")

In [ ]:
sm_2 = pystan.StanModel(file="2-steps.stan")

In [ ]:
sm_const = pystan.StanModel(file="const.stan")

In [ ]:
# Train the model and generate samples
fit_const = sm_const.sampling(data=data, iter=iteration, init='random')
fit_const

In [ ]:
# Train the model and generate samples
fit_start_1 = sm_1.sampling(data=data, iter=iteration, init='random')
fit_start_1

In [ ]:
# Train the model and generate samples
fit_start_2 = sm_2.sampling(data=data, iter=iteration, init='random')
fit_start_2

In [ ]:
### Convergence

In [ ]:
data_const = az.from_pystan(fit_const, log_likelihood='log_lik')
data_1 = az.from_pystan(fit_start_1, log_likelihood='log_lik')
data_2 = az.from_pystan(fit_start_2, log_likelihood='log_lik')

In [ ]:
az.plot_trace(data_const, var_names=['init_inf', 'b', 'a', 'd', 'q'])

In [ ]:
az.plot_trace(data_1, var_names=['init_inf', 'b0', 'b1', 'b_date', 'theta_b', 'q0', 'q1', 'q_date', 'theta_q', 'a', 'd'])

In [ ]:
az.plot_trace(data_2, var_names=['init_inf', 'b0', 'b1', 'b2', 'b_date', 'b2_date', 'theta_b', 'theta_b2', 
                                 'q0', 'q1', 'q2', 'q_date', 'q2_date', 'theta_q', 'theta_q2', 'a', 'd'])

In [ ]:
### model selection

In [ ]:
loo_const = az.loo(data_const, pointwise=True)
loo_const

In [ ]:
loo_const.loc['pareto_k']

In [ ]:
loo_const.loc['loo_i']

In [ ]:
loo_1 = az.loo(data_1, pointwise=True)
loo_1

In [ ]:
loo_1['pareto_k'][55:]

In [ ]:
loo_1['loo_i']

In [ ]:
loo_i = pd.DataFrame({'const':loo_const['loo_i'], '1-step':loo_1['loo_i']})
sns.relplot(data=loo_i)

In [ ]:
pareto_k = pd.DataFrame({'const':loo_const['pareto_k'], '1-step':loo_1['pareto_k']})
sns.relplot(data=pareto_k)

In [ ]:
az.compare({'const':data_const, '1-step':data_1})

In [ ]:
data_CV = {'T': C0.shape[0], 'T0': 55, 'P': P, 'C0': C0, 'R0': R0, 'D0':D0}

In [ ]:
# Train the model and generate samples
fit_1_CV = sm_1.sampling(data=data_CV, iter=iteration, init='random')
fit_1_CV

In [ ]:
# Train the model and generate samples
fit_const_CV = sm_const.sampling(data=data_CV, iter=iteration, init='random')
fit_const_CV

In [ ]:
fit_1_CV['v_log_lik'].mean()

In [ ]:
fit_const_CV['v_log_lik'].mean()

In [ ]:
data_CV_const = az.from_pystan(fit_const_CV, log_likelihood='log_lik')
data_CV_1 = az.from_pystan(fit_1_CV, log_likelihood='log_lik')

In [ ]:
az.compare({'const':data_CV_const, '1-step':data_CV_1})

In [ ]:
### Visualization ###

In [ ]:
def sigmoid(x, theta, t0):
    return 1/(1 + np.exp(-theta*(x-t0)))

In [ ]:
def simulate(init_inf, b0, b1, theta_b, b_date, q0, q1, theta_q, q_date,
             a, d, horizon):
    S = init_inf;
    R = 0;
    D = 0;
    S_list = [S]
    R_list = [R]
    D_list = [D]
    b_list = [b0]
    q_list = [0]
    q = q0 + (q1 - q0) * sigmoid(0, theta_q, q_date)
    S0 = [scipy.stats.poisson.rvs(q*init_inf)]
    R0 = [scipy.stats.poisson.rvs(a * S0[0])]
    D0 = [scipy.stats.poisson.rvs(d * S0[0])]
    for t in range(horizon):
        b = b0 + (b1-b0) * sigmoid(t, theta_b, b_date)
        I = (S - R - D) * b * (1 - S/P)
        q = q0 + (q1 - q0) * sigmoid(t, theta_q, q_date)
        NR = a * (S - R - D)
        ND = d * (S - R - D)
        D = D + ND
        S = S + I
        R = R + NR
        S_list.append(S)
        R_list.append(R)
        D_list.append(D)
        b_list.append(b)
        q_list.append(q)
        NI0 = scipy.stats.poisson.rvs(q*I)
        NR0 = scipy.stats.poisson.rvs(a * (S0[-1] - R0[-1] - D0[-1]))
        ND0 = scipy.stats.poisson.rvs(d * (S0[-1] - R0[-1] - D0[-1]))
        S0.append(S0[-1]+NI0)
        R0.append(R0[-1]+NR0)
        D0.append(D0[-1]+ND0)
    data = pd.DataFrame({'S':np.array(S_list),
                        'R':np.array(R_list),
                        'D':np.array(D_list),
                        'b':np.array(b_list),
                        'q':np.array(q_list),
                        'S0':np.array(S0),
                        'R0':np.array(R0),
                        'D0':np.array(D0)},
                        index=pd.date_range(start=epoch, periods=horizon+1, freq='D'))
    return data

In [ ]:
def monte_carlo(fit, horizon):
    S = pd.DataFrame()
    R = pd.DataFrame()
    D = pd.DataFrame()
    b = pd.DataFrame()
    q = pd.DataFrame()
    S0 = pd.DataFrame()
    R0 = pd.DataFrame()
    I0 = pd.DataFrame()
    D0 = pd.DataFrame()
    n = fit['init_inf'].shape[0]
    for k in range(repeat):
        i = np.random.randint(n)
        try:
            q0 = fit['q0'][i]
        except:
            q0 = 0
        
        df = simulate(init_inf=fit['init_inf'][i],
                      b0=fit['b0'][i],
                      b1=fit['b1'][i],
                      theta_b=fit['theta_b'][i],
                      b_date=fit['b_date'][i],
                      q0=q0,
                      q1=fit['q1'][i],
                      theta_q=fit['theta_q'][i],
                      q_date=fit['q_date'][i],
                      a=fit['a'][i],
                      d=fit['d'][i],
                      horizon=horizon)
        S = pd.concat([S, df['S']], axis=1)
        R = pd.concat([R, df['R']], axis=1)
        D = pd.concat([D, df['D']], axis=1)
        b = pd.concat([b, df['b']], axis=1)
        q = pd.concat([q, df['q']], axis=1)
        S0 = pd.concat([S0, df['S0']], axis=1)
        R0 = pd.concat([R0, df['R0']], axis=1)
        D0 = pd.concat([D0, df['D0']], axis=1)
        I0 = pd.concat([I0, df['S0'] - df['R0']], axis=1)
    return S, R, D, b, q, S0, R0, I0, D0
    

In [ ]:
def extract_series(fit, name):
    value_list = []
    for d in range(S0.shape[0]):
        vals = fit['{}[{}]'.format(name, d+1)]
        value_list.append(vals)
    data = np.array(value_list)
    return pd.DataFrame(data, 
                        index=confirmed.index)

In [ ]:
def draw(simulated, real=None, upto=pd.to_datetime('2020-04-18')):
    graph=pd.DataFrame(index=pd.date_range(start=epoch, end=upto))
    simulated = simulated.dropna(axis=1)
    median = simulated.median(axis=1)
    upper = simulated.quantile(q=0.75, axis=1)
    lower = simulated.quantile(q=0.25, axis=1)
    if not real is None:
        graph['Real'] = real
    graph['Median'] = median
    graph['Upper'] = upper
    graph['Lower'] = lower
    sns.relplot(kind="line", data=graph, aspect=2)

In [ ]:
S, R, D, b, q, S0, R0, I0, D0 = monte_carlo(fit_start_1, 120)

In [ ]:
draw(I0, real=(confirmed-recovered)[country])

In [ ]:
S.median(axis=1)[pd.to_datetime('2020-04-18')]

In [ ]:
draw(q)

In [ ]:
draw(b)

In [ ]:
draw(R0, real=recovered[country])

In [ ]:
draw(D0, real=death[country])

In [ ]:
draw(S, upto=pd.to_datetime('2020-05-30'))

In [ ]:
draw(D, upto=pd.to_datetime('2020-05-30'))

In [ ]:
S.loc[pd.to_datetime('2020-05-30')]

In [ ]:
## Saving results

In [ ]:
import pickle
with open("sm1_fit.pkl", "wb") as f:
    pickle.dump({'model' : sm_1, 'fit' : fit_start_1}, f, protocol=-1)
    # or with a list
    # pickle.dump([model, fit], f, protocol=-1)

In [ ]:
import pickle
with open("sm2_fit.pkl", "wb") as f:
    pickle.dump({'model' : sm_2, 'fit' : fit_start_2}, f, protocol=-1)
    # or with a list
    # pickle.dump([model, fit], f, protocol=-1)

In [ ]:
import pickle
with open("sm_const_fit.pkl", "wb") as f:
    pickle.dump({'model' : sm_const, 'fit' : fit_const}, f, protocol=-1)
    # or with a list
    # pickle.dump([model, fit], f, protocol=-1)

In [ ]:
## Validation by simulation

In [ ]:
i = 1
fit = fit_start_1
horizon=(pd.to_datetime('2020-04-18') - pd.to_datetime('2020-02-01')).days
df = simulate(init_inf=fit['init_inf'][i],
                      b0=fit['b0'][i],
                      b1=fit['b1'][i],
                      theta_b=fit['theta_b'][i],
                      b_date=fit['b_date'][i],
                      q0=fit['q0'][i],
                      q1=fit['q1'][i],
                      theta_q=fit['theta_q'][i],
                      q_date=fit['q_date'][i],
                      a=fit['a'][i],
                      d=fit['d'][i],
                      horizon=horizon)
df['S0']

In [ ]:
C0 = df['S0'].values
R0 = df['R0'].values
D0 = df['D0'].values
data_simulated = {'T': C0.shape[0], 'P': P, 'C0': C0, 'R0': R0, 'D0':D0}

In [ ]:
# Train the model and generate samples
fit_simulated = sm_1.sampling(data=data_simulated, iter=iteration, init=0)
fit_simulated

In [ ]:
(fit_start_1['init_inf'][i], fit_start_1['b0'][i], fit_start_1['b1'][i], fit_start_1['theta_b'][i], 
fit_start_1['b_date'][i], fit_start_1['q0'][i], fit_start_1['q1'][i], fit_start_1['theta_q'][i], fit_start_1['q_date'][7]
,fit_start_1['a'][i], fit_start_1['d'][i])